<a href="https://colab.research.google.com/github/oumaymadahmani/ML/blob/main/TP3_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Chargement de fichier movie_review.csv**

In [ ]:
import pandas as pd
# Charger le fichier CSV depuis le stockage local
from google.colab import files
uploaded = files.upload()
# Charger le fichier CSV en utilisant pandas
df = pd.read_csv("movie_review.csv")
# Afficher les premières lignes du DataFrame pour vérification
print(df.head())

Saving movie_review.csv to movie_review (1).csv
   fold_id cv_tag  html_id  sent_id  \
0        0  cv000    29590        0   
1        0  cv000    29590        1   
2        0  cv000    29590        2   
3        0  cv000    29590        3   
4        0  cv000    29590        4   

                                                text  tag  
0  films adapted from comic books have had plenty...  pos  
1  for starters , it was created by alan moore ( ...  pos  
2  to say moore and campbell thoroughly researche...  pos  
3  the book ( or " graphic novel , " if you will ...  pos  
4  in other words , don't dismiss this film becau...  pos  


In [ ]:
df

,fold_id,cv_tag,html_id,sent_id,text,tag
0,0,cv000,29590,0,films adapted from comic books have had plenty...,pos
1,0,cv000,29590,1,"for starters , it was created by alan moore ( ...",pos
2,0,cv000,29590,2,to say moore and campbell thoroughly researche...,pos
3,0,cv000,29590,3,"the book ( or "" graphic novel , "" if you will ...",pos
4,0,cv000,29590,4,"in other words , don't dismiss this film becau...",pos
...,...,...,...,...,...,...
64715,9,cv999,14636,20,that lack of inspiration can be traced back to...,neg
64716,9,cv999,14636,21,like too many of the skits on the current inca...,neg
64717,9,cv999,14636,22,"after watching one of the "" roxbury "" skits on...",neg
64718,9,cv999,14636,23,"bump unsuspecting women , and . . . that's all .",neg


In [ ]:
x=df['text']
x

0        films adapted from comic books have had plenty...
1        for starters , it was created by alan moore ( ...
2        to say moore and campbell thoroughly researche...
3        the book ( or " graphic novel , " if you will ...
4        in other words , don't dismiss this film becau...
                               ...                        
64715    that lack of inspiration can be traced back to...
64716    like too many of the skits on the current inca...
64717    after watching one of the " roxbury " skits on...
64718     bump unsuspecting women , and . . . that's all .
64719    after watching _a_night_at_the_roxbury_ , you'...
Name: text, Length: 64720, dtype: object

In [ ]:
y=df['tag']
y

0        pos
1        pos
2        pos
3        pos
4        pos
        ... 
64715    neg
64716    neg
64717    neg
64718    neg
64719    neg
Name: tag, Length: 64720, dtype: object

In [ ]:
!pip install nltk

import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Télécharger les ressources NLTK nécessaires
nltk.download('stopwords')# mots vides
nltk.download('punkt')
# Prétraitement des données textuelles
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    text = re.sub(r'[^\w\s]', '', text)  # Supprimer la ponctuation (re:expression réguliere ^\w\s: ni caractere alphanumerique ni caractere d'espace :effacer tout char diff de alphanumerique et espace)
    tokens = word_tokenize(text.lower())  # Convertir en minuscules et tokeniser
    filtered_tokens = [word for word in tokens if word not in stop_words]  # Supprimer les stop words
    return filtered_tokens

df['text'] = df['text'].apply(preprocess_text)

# Entraînement du modèle Word2Vec
model = Word2Vec(sentences=df['text'])

# Vectorisation des critiques de films
def vectorize_text(text):
    vectors = [model.wv[word] for word in text if word in model.wv] #extraire le vecteur des mots a cdts qu'ils existent dans le vocabulaire Word2vec
    if vectors:
        return np.mean(vectors, axis=0)#si vectors n'est pas vide la fct return la moyenne des vecteurs ds vectors
    else:
        return np.zeros(model.vector_size)#sinon elle return un vecteur nul avec la meme dimensiion du vocabulaire

df['Vectorized_Text'] = df['text'].apply(vectorize_text)

# Division des données
X_train, X_test, y_train, y_test = train_test_split(df['Vectorized_Text'].to_numpy(),
                                                   df['tag'].to_numpy(),
                                                    test_size=0.2,)# .tonumpy():convertir une structure/un ensemble de données en un tableau numpy car trai_test_split necessite des tableaux comme parametre

# Construction d'un classificateur
#  la régression logistique comme exemple
classifier = LogisticRegression()
classifier.fit(X_train.tolist(), y_train)

# Évaluation du modèle
y_pred = classifier.predict(X_test.tolist())

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Accuracy: 0.5713071693448702
Precision: 0.5733163827166847
Recall: 0.5713071693448702
F1 Score: 0.5654556178786158
